# Thorough Comparison of Reconstruction Methods

**Author:** Jakub Walczak, PhD

The analysis relies on the experiments' results collected for three investigated methods: OK, IDW, and SiNET.
For each out of 100 samples the hyper-parameter optimisation was carried out. Having optimised hyperparameters, 
the evaluation of methods on test test was performed.

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

In [ ]:
NOTEBOOK_DIR = Path(__session__).parent.parent

In [ ]:
RESULTS_DIR = NOTEBOOK_DIR / "results"
OK_RESULTS = RESULTS_DIR / "ok"
SINET_RESULTS = RESULTS_DIR / "inr" / "sinet"
MMGN_RESULTS = RESULTS_DIR / "inr" / "mmgn"
IDW_RESULTS = RESULTS_DIR / "idw"

## 1. Metrics Comparison

In [ ]:
ok_data = pd.read_csv(OK_RESULTS / "metrics.csv").set_index("dataset_id").add_suffix("_ok", axis=1)
idw_data = pd.read_csv(IDW_RESULTS / "metrics.csv").drop("Unnamed: 0", axis=1).set_index("dataset_id").add_suffix("_idw", axis=1)
mmgn_data = pd.read_csv(MMGN_RESULTS / "metrics.csv").set_index("dataset_id").add_suffix("_mmgn", axis=1)
sinet_data = pd.read_csv(SINET_RESULTS / "metrics.csv").set_index("dataset_id").add_suffix("_sinet", axis=1)

In [ ]:
all_data = pd.merge(
    idw_data,
    ok_data,
    how="inner",
    on="dataset_id",
)
all_data = pd.merge(
    all_data,
    mmgn_data,
    how="inner",
    on="dataset_id"
)
all_data = pd.merge(
    all_data,
    sinet_data,
    how="inner",
    on="dataset_id"
)
all_data.head()

In [ ]:
order = ["idw", "ok", "mmgn", "sinet"]
rmse_best_idx = np.argmin(
    all_data[["RMSE_idw", "RMSE_ok", "RMSE_mmgn", "RMSE_sinet"]], axis=1
)
r2_best_idx = np.argmax(all_data[["R^2_idw", "R^2_ok", "R^2_mmgn", "R^2_sinet"]], axis=1)
mabs_best_idx = np.argmin(
    all_data[["Max Abs Error_idw", "Max Abs Error_ok", "Max Abs Error_mmgn", "Max Abs Error_sinet"]],
    axis=1,
)

all_data["best_RMSE"] = [order[i] for i in rmse_best_idx]
all_data["best_R^2"] = [order[i] for i in r2_best_idx]
all_data["best_Max Abs Error"] = [order[i] for i in mabs_best_idx]

In [ ]:
all_data = all_data[
    [
        "RMSE_idw",
        "RMSE_ok",
        "RMSE_mmgn",
        "RMSE_sinet",
        "best_RMSE",
        "Max Abs Error_idw",
        "Max Abs Error_ok",
        "Max Abs Error_mmgn",
        "Max Abs Error_sinet",
        "best_Max Abs Error",
        "R^2_idw",
        "R^2_ok",
        "R^2_mmgn",
        "R^2_sinet",
        "best_R^2",
    ]
]

In [ ]:
all_data

## 2. Hyperparameters analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("dark")

### 2a. Ordinary Kriging

In [ ]:
ok_hparams = pd.read_csv(OK_RESULTS / "hparams_summary.csv")

In [ ]:
ok_hparams = ok_hparams.drop("dataset_id", axis=1)
ok_hparams.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ok_hparams["nlags"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("No. of lags");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ok_hparams["anisotropy_scaling"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Anisotropy scaling factor");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ok_hparams.groupby("coordinates_type").count()["nlags"].plot.bar(ax=ax)
ax.set_xlabel("Coordinate type");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ok_hparams.groupby("variogram_model").count()["nlags"].plot.bar(ax=ax)
ax.set_xlabel("Variogram model");

### 2b. IDW

In [ ]:
idw_hparams = pd.read_csv(IDW_RESULTS / "hparams_summary.csv")

In [ ]:
idw_hparams = idw_hparams.drop("dataset_id", axis=1)
idw_hparams.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
idw_hparams["k"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("No. of neighbours");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
idw_hparams["k_min"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Min. no. of neighbours");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
idw_hparams["power"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Power factor");

### 2c. MMGN

In [ ]:
mmgn_hparams = pd.read_csv(MMGN_RESULTS / "hparams_summary.csv")

In [ ]:
mmgn_hparams.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["lr"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Learning rate");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["weight_decay"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Weight decay");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["hidden_dim"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Hidden dim");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["latent_dim"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Latent dim");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["n_layers"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Number of layers");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["input_scale"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Input scale");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
mmgn_hparams["alpha"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Alpha");

### 2d. SiNET

In [ ]:
sinet_hparams = pd.read_csv(SINET_RESULTS / "hparams_summary.csv")

In [ ]:
sinet_hparams.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["lr"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Learning rate");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["weight_decay"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Weight decay");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["scale"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Scale");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["num_epochs"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("No. of epochs");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["batch_size"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Batch size");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["hidden_dim"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Hidden dim");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["mse_loss_weight"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("MSE loss weight");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["eikonal_loss_weight"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Eikonal loss weight");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
sinet_hparams["laplace_loss_weight"].plot.hist(bins=10, ax=ax)
ax.set_xlabel("Laplace loss weight");